# Cerberus

The watch-dog of Hades, whose duty it was to guard the entrance; everybody, sooner or later, had to go there, and nobody wanted to carry off the entrance.

Cerberus provides powerful yet simple and lightweight data validation functionality out of the box and is designed to be easily extensible, allowing for custom validation.

In [152]:
# Importando o cerberus e pandas
from cerberus import Validator
import pandas as pd
import copy
import numpy as np

O `df_certo` é o df em que todas as validações devem passar. Já o `df_errado` é o df em que teremos problemas em alguns testes. Mas, o Cerberus trabalha com dados em formato de dicionário, então vamos transformar os dois dataframes no `json_certo` e no `json_errado`.
 
Lista de validações que realizaremos:
1. Verificar se `id_compra` é único
2. Verificar se todas as datas em `data` está corretas
3. Verificar se `ano_mes_id` está coerente com `data`
4. Verificar se `ano` está coerente com `data`
5. Verificar se `mes` está coerente com `data`
6. Verificar se `dia` está coerente com `data`
7. Verificar se o valor `total` é a soma de `valor` e `frete`
8. Verificar se o `produto` faz parte do catálogo
9. Verificar se os tipos de cada coluna estão coerentes

In [139]:
# Importando os dados
df_certo = pd.read_excel("exemplo_validacao_certo.xlsx", usecols = "A:K")
df_errado = df_certo.copy(deep=True)
json_certo = df_certo.to_dict(orient='list')
json_errado = df_errado.to_dict(orient='list')

In [161]:
# Criando estrutura de validação e validador
schema = {  'usuario': {'type': ['string', 'list'], 'required': True},
            'id_compra': {'type': ['integer', 'list'], 'required': True},
            'data': {'type': ['date', 'list'], 'required': True, 'check_with' : _check_ano_mes_dia},
            'ano_mes_id' : {'type': ['integer', 'list'], 'required': True},
            'ano' : {'type': ['integer', 'list'], 'required': True},
            'mes' : {'type': ['integer', 'list'], 'required': True},
            'dia' : {'type': ['integer', 'list'], 'required': True},
            'produto' : {'type': ['string', 'list'], 'required': True, 'allowed': ["capinha","tablet","pelicula",
                                                                                    "cabo","celular", "mouse", "teclado",
                                                                                    "computador","fone","monitor"]},
            'valor' : {'type': ['float', 'list'], 'required': True},
            'frete' : {'type': ['float', 'list'], 'required': True},
            'total' : {'type': ['float', 'list'], 'required': True, 'check_with' :_check_valor_total}
            }
validador = Validator(schema)

In [162]:
# Verificando a planilha correta
validador.validate(json_certo)

True

In [129]:
# Comando para verificar erros
print(validador.errors)

{}


In [131]:
# Removendo coluna do df_errado
del json_errado['usuario']
validador.validate(json_errado)
# Verificando erros identificados
print(validador.errors)
# Resetando o json
json_errado = copy.deepcopy(json_certo)

{'usuario': ['required field']}


In [132]:
# Inserindo produto não catalogado
json_errado['produto'][15] = 'caneta'
validador.validate(json_errado)
# Verificando erros identificados
print(validador.errors)
# Resetando o json
json_errado = copy.deepcopy(json_certo)

{'produto': ["unallowed values ('caneta',)"]}


O cerberus não parece possuir uma forma de validar valores únicos numa coluna. Podemos usar o validador de JSON no lugar.

In [133]:
# Importando validador
from jsonschema import validate
# Criando validação
schema_unique = { "type": "integer", "uniqueItems": True}
# Inserindo erro
json_errado['id_compra'][0] = 1
json_errado['id_compra'][1] = 1
# Validando erro
try:
    validate(instance = json_errado['id_compra'], schema = schema_unique)
except Exception as e:
    print(e)
# Resetando json
json_errado = copy.deepcopy(json_certo)

[1, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50] is not of type 'integer'

Failed validating 'type' in schema:
    {'type': 'integer', 'uniqueItems': True}

On instance:
    [1,
     1,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50]


In [126]:
# Definindo função para checar se ano, mes e dia estão coerentes
def _check_ano_mes_dia(field, value, error):
    if [linha.year for linha in value] != json_errado['ano']:
        error(field, 'Ano não bate')
    if [linha.month for linha in value] != json_errado['mes']:
        error(field, 'Mês não bate')
    if [linha.day for linha in value] != json_errado['dia']:
        error(field, 'Dia não bate')

In [134]:
# Causando erro de ano
json_errado['ano'][41] = 2021
# Validando erro
validador.validate(json_errado)
print(validador.errors)
# Resetando json
json_errado = copy.deepcopy(json_certo)

{'data': ['Ano não bate']}


In [160]:
# Definindo função que certifica valor total
def _check_valor_total(field, value, error):
    if value != [x + y for x, y in zip(json_errado['valor'], json_errado['frete'])]:
        error(field, 'Valor total errado, possível erro nos valores de frete ou valor do produto')

In [163]:
# Causando erro de valores
json_errado['total'][34] = 0.00
# Validando erro
validador.validate(json_errado)
print(validador.errors)
# Resetando json
json_errado = copy.deepcopy(json_certo)

{'total': ['Valor total errado, possível erro nos valores de frete ou valor do produto']}
